# 写真に映る動物が犬か猫かを分類

In [ ]:
import tensorflow as tf

# データセットの読み込み
train_dataset = tf.keras.preprocessing.image_dataset_from_directory(
    "dog_cat_photos/train",
    image_size=(96, 96),
    label_mode="binary",
    batch_size=32,
    shuffle=True
)

test_dataset = tf.keras.preprocessing.image_dataset_from_directory(
    "dog_cat_photos/test",
    image_size=(96, 96),
    label_mode="binary",
    batch_size=32,
    shuffle=False
)

Found 300 files belonging to 2 classes.
Found 100 files belonging to 2 classes.


In [4]:
# 画像の水増し用関数
def flip_left_right(image, label):
    image = tf.image.random_flip_left_right(image)
    return image, label

def flip_up_down(image, label):
    image = tf.image.random_flip_up_down(image)
    return image, label

def rot90(image, label):
    image = tf.image.rot90(image)
    return image, label

def rot180(image, label):
    image = tf.image.rot90(image, k=2)
    return image, label

def rot270(image, label):
    image = tf.image.rot90(image, k=3)
    return image, label

# 水増しの実行
train_dataset_lr = train_dataset.map(flip_left_right)
train_dataset_ud = train_dataset.map(flip_up_down)
train_dataset_r90 = train_dataset.map(rot90)
train_dataset_r180 = train_dataset.map(rot180)
train_dataset_r270 = train_dataset.map(rot270)

# 水増しデータセットの結合
train_dataset = (
    train_dataset
    .concatenate(train_dataset_lr)
    .concatenate(train_dataset_ud)
    .concatenate(train_dataset_r90)
    .concatenate(train_dataset_r180)
    .concatenate(train_dataset_r270)
)

# シャッフル
train_dataset = train_dataset.shuffle(32)

In [9]:
# モデルの作成
ipnut_layer = tf.keras.layers.Input(shape=(96, 96, 3))
l_layer = tf.keras.applications.mobilenet_v2.preprocess_input(ipnut_layer)

base_model = tf.keras.applications.mobilenet_v2.MobileNetV2(
    input_shape=(96, 96, 3),
    input_tensor=l_layer,
    include_top=False,
    weights="mobilenet_v2_weights_tf_dim_ordering_tf_kernels_1.0_96_no_top.h5",
    pooling='avg'
)
base_model.trainable = False

output_layer = tf.keras.layers.Dense(1, activation='sigmoid')

model = tf.keras.models.Sequential([
    base_model,
    output_layer
])
model.compile(optimizer='adam',
              loss='binary_crossentropy',
              metrics=['accuracy'])

In [10]:
# モデルの学習
model.fit(train_dataset, epochs=20)

Epoch 1/20
60/60 ━━━━━━━━━━━━━━━━━━━━ 3s 29ms/step - accuracy: 0.8267 - loss: 0.3824
Epoch 2/20
60/60 ━━━━━━━━━━━━━━━━━━━━ 2s 29ms/step - accuracy: 0.9328 - loss: 0.1809
Epoch 3/20
60/60 ━━━━━━━━━━━━━━━━━━━━ 2s 29ms/step - accuracy: 0.9578 - loss: 0.1291
Epoch 4/20
60/60 ━━━━━━━━━━━━━━━━━━━━ 2s 30ms/step - accuracy: 0.9694 - loss: 0.1028
Epoch 5/20
60/60 ━━━━━━━━━━━━━━━━━━━━ 2s 28ms/step - accuracy: 0.9806 - loss: 0.0834
Epoch 6/20
60/60 ━━━━━━━━━━━━━━━━━━━━ 2s 30ms/step - accuracy: 0.9850 - loss: 0.0707
Epoch 7/20
60/60 ━━━━━━━━━━━━━━━━━━━━ 2s 32ms/step - accuracy: 0.9900 - loss: 0.0596
Epoch 8/20
60/60 ━━━━━━━━━━━━━━━━━━━━ 2s 32ms/step - accuracy: 0.9906 - loss: 0.0526
Epoch 9/20
60/60 ━━━━━━━━━━━━━━━━━━━━ 2s 31ms/step - accuracy: 0.9922 - loss: 0.0463
Epoch 10/20
60/60 ━━━━━━━━━━━━━━━━━━━━ 2s 30ms/step - accuracy: 0.9928 - loss: 0.0448
Epoch 11/20
60/60 ━━━━━━━━━━━━━━━━━━━━ 2s 32ms/step - accuracy: 0.9944 - loss: 0.0379
Epoch 12/20
60/60 ━━━━━━━━━━━━━━━━━━━━ 2s 34ms/step - accuracy:

In [13]:
# テストデータで分類
pred_data = model.predict(test_dataset)

4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 30ms/step


In [14]:
# モデルの評価
model.evaluate(test_dataset)

4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 32ms/step - accuracy: 0.9800 - loss: 0.0405


[0.04049813747406006, 0.9800000190734863]